# Data Generation for quadcopter system with various obstacles (sphere, capsule, box)

In [1]:
import sys
import os
import random
import math
import logging
import time
from tqdm.notebook import tqdm

sys.path.append('../lib')

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
from IPython.display import clear_output
import time
from ocp import *
from costs import *
from ocp_utils import *
from env_creator import EnvCreator, generate_sdf_rep
from tensor_decomp import apply_tt
from visualization_utils import plot_traj_projections, plot_traj_and_obs_3d

import pybullet as p
import pybullet_data

import trimesh

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

## Steps of trajectory optimization 

1. Define the dynamical system (e.g., double integrator, 2d robot, n-d manipulator, etc.)
2. Define the cost functions
3. Construct the ILQR problem
4. Solve

## Pybullet

In [2]:
p.connect(p.GUI)
# p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [3]:
p.resetSimulation()

## iLQR functions 

### Setup the ILQR & Solve

In [4]:
### Define system and timesteps 

T = 100 # number of timesteps 
Dx, Du = 12, 4 #dimensions of x and u

x0 = np.array([-1,-1,-1,0,0,0,0,0,0,0,0,0])
x_target = np.array([1, 1, 1, 0,0, 0,0,0,0,0,0,0])
x_w = np.array([0,0,-1,0,0,0,0,0,0,0,0,0])
print("Start config: {}\nTarget config: {}\nVia: {}".format(x0, x_target, x_w))

# Define obstacles 
obs1 = {
    'pos': np.array([-0.3, -0.1, 0.2]), # obstacle position
    'rad': 0.45,                     # radius of the obstacle 
    'length':0.3,
    'w': 100.,                       # weight of obstacle cost
    'd_marg': 0.1,                    # margin of obstacle
    'obs_type':p.GEOM_CAPSULE
    }

obs2 = {'pos': np.array([0.2, 0.2, -0.2]), 
        'halfExtents': [0.4,0.4,0.1], 
        'w': 100.,   
        'd_marg': 0.1  ,
        'obs_type':p.GEOM_BOX
    }

obstacles = [obs1, obs2]

# set ilqr params 
max_iterations = 100

Start config: [-1 -1 -1  0  0  0  0  0  0  0  0  0]
Target config: [1 1 1 0 0 0 0 0 0 0 0 0]
Via: [ 0  0 -1  0  0  0  0  0  0  0  0  0]


In [9]:
p.resetSimulation()

#### Create cost function between Sphere to any object

In [10]:
x0 = np.array([-1,-1,0.3,0,0,0,0,0,0,0,0,0])
x_target = np.array([1,1,.7,0,0,0,0,0,0,0,0,0])

quad_id, obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet_quadcopter(x0, x_target, obstacles)

prob, robot_sys = setup_ilqr_quadcopter_general(100, x0, x_target, obstacles, obj_id)

p.resetBasePositionAndOrientation(obj_id, x0[:3], (0,0,0,1))

#### Create initialisation & solve iLQR

In [11]:
xs_init, us_init = create_standard_init(robot_sys, x0, T)

xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=max_iterations, th_grad = 1e-9, th_stop = 1e-9)

In [12]:
# plot_traj(xs, obj_id, dt = 0.04)
robot_sys.vis_traj(quad_id, xs, changeCamera = False, dt = 0.05)

***

## Data generation 

#### Randomize the init & goal positions

In [13]:
# include check that start and goal don't coincide / have a certain distance to eachother 
xrange1 = (-1., -0.3)
yrange1 = (-1., -0.3)
zrange1 = (-1., -0.3)

xrange2 = (0.3, 1.)
yrange2 = (0.3, 1.)
zrange2 = (0.3, 1.)

num_samples = 100
init_pos = []
goal_pos = []

for i in range(num_samples): 
    init_pos.append(np.array([random.uniform(*xrange1), random.uniform(*yrange1), random.uniform(*zrange1),0,0,0])) 
    goal_pos.append(np.array([random.uniform(*xrange2), random.uniform(*yrange2), random.uniform(*zrange2),0,0,0]))

#### Randomize obstacles

In [14]:
# adapt obstacle parameters 
xrange4 = (-0.4, 0.4)
yrange4 = (-0.4, 0.4)
zrange4 = (-0.4, 0.4)


num_positions = 1000
obs_positions4 = []
[obs_positions4.append(np.array(
    [random.uniform(*xrange4), 
     random.uniform(*yrange4), 
     random.uniform(*zrange4)])) 
 for i in range(num_positions) ]

obs_positions4 = np.array(obs_positions4)
obs_positions = [obs_positions4]

In [15]:
#for multiple obstacle
num_radii = 5 
num_length = 5
num_length_half = 5
radii = np.linspace(0.3, 0.5, num_radii)
lengths = np.linspace(0.3, 0.5, num_length)
rad_capsules = np.linspace(0.3, 0.5, num_radii)
x_length = np.linspace(0.3, 0.5, num_length_half)
y_length = np.linspace(0.3, 0.5, num_length_half)
z_length = np.linspace(0.1, 0.5, num_length_half)
half_extents = np.vstack([x_length, y_length, z_length]).T

#### Pick an obstacle randomly

In [16]:
def gen_random_obstacle(n=1):
    #types: spheres:0, box:1, capsule:2
    obstacles = []
    for i in range(n):
        obstacle = {'w':100., 'd_marg':0.20}

        #randomize pos
        idx = np.random.randint(len(obs_positions[0]))
        pos = obs_positions[0][idx]
        obstacle['pos'] = pos

        obs_type = np.random.randint(3)
        if obs_type == 0:
            rad = np.random.choice(radii)
            obstacle['rad'] = rad
            obstacle['obs_type'] = p.GEOM_SPHERE
        elif obs_type == 1:
            idx = np.random.randint(len(half_extents))
            obstacle['halfExtents'] = half_extents[idx]
            obstacle['obs_type'] = p.GEOM_BOX
        elif obs_type ==2:
            rad = np.random.choice(rad_capsules)
            length = np.random.choice(lengths)
            obstacle['rad'] = rad
            obstacle['length'] = length
            obstacle['obs_type'] = p.GEOM_CAPSULE

        obstacles += [obstacle]
    return obstacles
# quad_id, obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet_quadcopter(x0, x_target, obstacles)

#### Solve ilqr

In [40]:
obstacles = gen_random_obstacle(3)
quad_id, obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet_quadcopter(x0, x_target, obstacles)

# Setup and solve problem 
prob, lin_sys = setup_ilqr_quadcopter_general(T, x0, x_target, obstacles, obj_id)

# get standard initialization
xs_init, us_init = create_standard_init(lin_sys, x0, T)

xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=max_iterations, th_grad = 1e-9, th_stop = 1e-9)

In [42]:
plot_traj(xs, obj_id)

***

### Batch Data Generation

In [16]:
# experiment_name = 'start_goal_fixed_100pos_5rad'
experiment_name = 'quad_mult_obs_var_start_goal_1000samples_three_obstacles'
log_file = '../../data/envs/%s/data_generation.txt'%experiment_name
os.makedirs(os.path.dirname(log_file), exist_ok=True)

w_obs = 100.   # weight of the obstacle cost
d_margin = 0.1 # margin of the obstacle
target_thres = 0.1 # does it reach target close enough -> target status (bool)
via_point = np.array([0,0,-1,0,0,0]) # bias 
success_track = []
data_set = []
exp_idx = 0

In [157]:
num_experiments = 2000
max_iterations = 100
for i in tqdm(range(num_experiments)): 
    if not fix_start_goal: 
        pos_idx = random.randint(0,len(init_pos)-1)
        x0 = init_pos[pos_idx]
        x_target = goal_pos[pos_idx]
    else: 
        x0 = np.array([-1., -1., -1., 0., 0., 0.])
        x_target = np.array([1., 1., 1., 0., 0., 0.])
    
    x0 = np.concatenate([x0, np.zeros(6)])
    x_target = np.concatenate([x_target, np.zeros(6)])
    
    exp_obstacles = gen_random_obstacle(3)
    quad_id, obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet_quadcopter(x0, x_target, exp_obstacles)
    
    col_status_init, _ = check_collision_general(exp_obstacles, obj_id, x0)
    col_status_goal, _ = check_collision_general(exp_obstacles, obj_id, x_target)
    
    if col_status_init or col_status_goal:
        print('Init/goal collides with obstacle')
        continue
    
    
    exp_idx += 1 

    # Setup and solve problem 
    prob, lin_sys = setup_ilqr_quadcopter_general(T, x0, x_target, exp_obstacles, obj_id)#     xs_init, us_init = create_standard_init(lin_sys, x0, T)
    xs_init, us_init = create_standard_init(lin_sys, x0, T)
    xs, us, ddp = solve_ilqr(prob, xs_init, us_init, iterations=max_iterations, th_grad = 1e-6, th_stop = 1e-6)
            
    # check status             
    status, target_status, collision_status = check_cost_general(xs, x_target, exp_obstacles, obj_id)
    status_info = 'Reach target: {}\nNo collision: {}\nSUCCESS: {}'.format(target_status, collision_status, status)
    logging.info(status_info)
    success_track.append(status)

    # save 
    data = dict()
    data['xs'] = np.array(ddp.xs)
    data['us'] = np.array(us)
    data['x0'] = x0
    data['xT'] = x_target
    data['x_w'] = via_point
    data['obstacles'] = exp_obstacles
    data['status'] = status
    data['target_status'] = target_status
    data['collision_status'] = collision_status
    data['cost'] = ddp.cost
    data['iter'] = ddp.iter
    data_set.append(data)
    np.save('../../data/envs/{}/data_exp{}.npy'.format(experiment_name,exp_idx), data)        

print('Success rate is {}'.format(np.sum(success_track)/(exp_idx)))